In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import layers, models


csv_path = '/content/drive/MyDrive/traffic_Data/labels.csv'
df = pd.read_csv(csv_path)

def load_images_and_labels(dataset_path):
    images = []
    labels = []

    for class_id, sign_name in zip(df['ClassId'], df['Name']):
        class_folder = os.path.join(dataset_path, str(class_id))
        images_paths = [os.path.join(class_folder, image_file) for image_file in os.listdir(class_folder)]
        images_paths = images_paths[:min(len(images_paths), 100)]
        for image_path in images_paths:
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (32, 32))
            images.append(img)
            labels.append(class_id)

    return np.array(images), np.array(labels)

dataset_path = '/content/drive/MyDrive/traffic_Data/DATA'
images, labels = load_images_and_labels(dataset_path)
images, labels = shuffle(images, labels)


X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


X_train, X_test = X_train / 255.0, X_test / 255.0


model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(58, activation='softmax')  # 43 classes for traffic signs
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")


model.save('road_sign_classifier_cnn.h5')





Epoch 1/5
75/75 [==============================] - 5s 60ms/step - loss: 3.6398 - accuracy: 0.0994 - val_loss: 3.0356 - val_accuracy: 0.2963
Epoch 2/5
75/75 [==============================] - 3s 36ms/step - loss: 2.6932 - accuracy: 0.3045 - val_loss: 2.0204 - val_accuracy: 0.4983
Epoch 3/5
75/75 [==============================] - 3s 36ms/step - loss: 1.6847 - accuracy: 0.5564 - val_loss: 1.2379 - val_accuracy: 0.6667
Epoch 4/5
75/75 [==============================] - 3s 37ms/step - loss: 1.0177 - accuracy: 0.7195 - val_loss: 0.8552 - val_accuracy: 0.7492
Epoch 5/5
75/75 [==============================] - 4s 57ms/step - loss: 0.7061 - accuracy: 0.8117 - val_loss: 0.6204 - val_accuracy: 0.8266
19/19 - 0s - loss: 0.6204 - accuracy: 0.8266 - 195ms/epoch - 10ms/step
Test accuracy: 0.8265992999076843


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
pip install googletrans==4.0.0-rc1


In [ ]:
from googletrans import Translator
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model


model_path = 'road_sign_classifier_cnn.h5'
model = load_model(model_path)

class_descriptions_path = '/content/drive/MyDrive/traffic_Data/labels.csv'
class_descriptions_df = pd.read_csv(class_descriptions_path)

def translate_description(description, dest_lang):
    translator = Translator()
    translated = translator.translate(description, dest=dest_lang)
    return translated.text

def predict_road_sign(image_path, model, class_descriptions_df):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (32, 32))
    img = img.reshape(1, 32, 32, 3)
    img = img / 255.0
    prediction = model.predict(img)
    sign_class = np.argmax(prediction)
    sign_description = class_descriptions_df[class_descriptions_df['ClassId'] == sign_class]['Name'].values[0]
    return sign_class, sign_description


image_path = '/content/speed_limti_70.png'
predicted_class, predicted_description = predict_road_sign(image_path, model, class_descriptions_df)
print(f"Predicted road sign class: {predicted_class}")

translated_description_english = translate_description(predicted_description, 'en')
print(f"Translated description (English): {translated_description_english}")


translated_description_kannada = translate_description(predicted_description, 'kn')
print(f"Translated description (Kannada): {translated_description_kannada}")

translated_description_marathi = translate_description(predicted_description, 'mr')
print(f"Translated description (Marathi): {translated_description_marathi}")

translated_description_japanese = translate_description(predicted_description, 'ja')
print(f"Translated description (Japanese): {translated_description_japanese}")

translated_description_hindi = translate_description(predicted_description, 'hi')
print(f"Translated description (Hindi): {translated_description_hindi}")

1/1 [==============================] - 0s 87ms/step
Predicted road sign class: 6
Translated description (English): Speed limit (70km/h)
Translated description (Kannada): ವೇಗದ ಮಿತಿ (ಗಂಟೆಗೆ 70 ಕಿ.ಮೀ.
Translated description (Marathi): वेग मर्यादा (70 किमी/ताशी)
Translated description (Japanese): 速度制限（70km/h）
Translated description (Hindi): गति सीमा (70 किमी/घंटा)
